# PPSP

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import log
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# Problem Formulation 

## Parameters
* $N$: number of projects to be selected
* $m$: limit of cardinality
* $r_i, r_{ij}, r_{ijk}$: benefit derived from implementing project $i$, $ij$ and $ijk$, respectively
* $d_i, d_{ij}, d_{ijk}$: required amount of resource by project $i$, $ij$ and $ijk$, respectively
* $b$: the available amount of resource

Note that the resources type may be multiple, but to simplify the question, we just choose one type to illustrate the resource costraint.  


Model formulations follow *An alternative efficient representation for the project portfolio selection problem*

In [2]:
def PPSP_solver(N, m, r_i, r_ij, r_ijk, d_i, d_ij, d_ijk, b, mode, equality=True):
    model = gp.Model('PPSP')
    model.setParam('OutputFlag', False)  # mute

    if mode == 'conventional':  # conventional models
        if equality == True:  # model 2-1, ECC
            x_list = []
            y_list = []
            z_list = []
            
            for i in range(N):
                x_list.append(i)
                for j in range(N):
                    if j > i:
                        y_list.append((i, j))
                    for k in range(N):
                        if j > i and k > j:
                            z_list.append((i, j, k))
                        
            
            x = model.addVars(x_list, vtype=GRB.BINARY, name='x')
            y = model.addVars(y_list, lb=0.0, ub=1, vtype=GRB.CONTINUOUS, name='y')
            z = model.addVars(z_list, lb=0.0, ub=1, vtype=GRB.CONTINUOUS, name='z')
            
            model.update()
                
            expr = gp.LinExpr()
            
            for i in range(N):
                expr.addTerms(r_i[i], x[i])
                
                for j in range(N):
                    if j > i:
                        expr.addTerms(r_ij[i, j], y[i, j])
                    
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(r_ijk[i, j, k], z[i, j, k])
                            
            model.setObjective(expr, GRB.MAXIMIZE)
            
            # Constraints
            
            ## C7
            expr = gp.LinExpr()
            for i in range(N):
                expr.addTerms(d_i[i], x[i])
                
                for j in range(N):
                    if j > i:
                        expr.addTerms(-d_ij[i, j], y[i, j])
                        
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(d_ijk[i, j, k], z[i, j, k])
            model.addLConstr(expr, GRB.LESS_EQUAL, b, 'C7')
            
            ## C8
            model.addLConstr(gp.quicksum(x[i] for i in range(N)) == m, 'C8')
            
                            
            ## C9
            for i in range(N):
                expr = gp.LinExpr()
                
                for j in range(N):
                    if j > i:
                        expr.addTerms(1, y[i, j])
                    elif j < i:
                        expr.addTerms(1, y[j, i])
                        
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(1, z[i, j, k])
                        elif i > j and k > i:
                            expr.addTerms(1, z[j, i, k])
                        elif k > j and i > k:
                            expr.addTerms(1, z[j, k, i])
                
                expr.addTerms(-m*(m-1)/2, x[i])
                model.addLConstr(expr, GRB.EQUAL, 0, f'C9_{i}')

        else:  # model 2-2, IECC
            x_list = []
            y_list = []
            z_list = []
            
            for i in range(N):
                x_list.append(i)
                for j in range(N):
                    if j > i:
                        y_list.append((i, j))
                    for k in range(N):
                        if j > i and k > j:
                            z_list.append((i, j, k))
                        
            
            x = model.addVars(x_list, vtype=GRB.BINARY, name='x')
            y = model.addVars(y_list, lb=0.0, ub=1, vtype=GRB.CONTINUOUS, name='y')
            z = model.addVars(z_list, lb=0.0, ub=1, vtype=GRB.CONTINUOUS, name='z')
            u = model.addVars(m, vtype=GRB.BINARY, name='u')
            phi = model.addVars(N, lb=0.0, vtype=GRB.CONTINUOUS, name='phi')
            bigM = m*(m-1)/2
            
            
            model.update()
                
            expr = gp.LinExpr()
            
            for i in range(N):
                expr.addTerms(r_i[i], x[i])
                
                for j in range(N):
                    if j > i:
                        expr.addTerms(r_ij[i, j], y[i, j])
                    
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(r_ijk[i, j, k], z[i, j, k])
                            
            model.setObjective(expr, GRB.MAXIMIZE)
            
            # Constraints
            ## C7
            expr = gp.LinExpr()
            for i in range(N):
                expr.addTerms(d_i[i], x[i])
                
                for j in range(N):
                    if j > i:
                        expr.addTerms(-d_ij[i, j], y[i, j])
                        
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(d_ijk[i, j, k], z[i, j, k])
            model.addLConstr(expr, GRB.LESS_EQUAL, b, 'C7')
            
            ## C10
            model.addLConstr(gp.quicksum(x[i] for i in range(N)) == gp.quicksum((t+1) * u[t] for t in range(m)), 'C10')
            
            ## C11
            model.addLConstr(gp.quicksum(u[t] for t in range(m)) == 1, 'C11')
            
            ## C12, C13
            for i in range(N):
                model.addLConstr(gp.quicksum(t*(t+1)/2*u[t] + bigM * (x[i]-1) for t in range(m)) <=  phi[i], f'C12_L_{i}')
                model.addLConstr(phi[i] <= gp.quicksum(t*(t+1)/2*u[t] + bigM * (1-x[i]) for t in range(m)), f'C12_R_{i}')
                model.addLConstr(phi[i] <= bigM * x[i], f'C13_{i}')
                
            ## C15
            for i in range(N):
                expr = gp.LinExpr()
                
                for j in range(N):
                    if j > i:
                        expr.addTerms(1, y[i, j])
                    elif j < i:
                        expr.addTerms(1, y[j, i])
                        
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(1, z[i, j, k])
                        elif j < i and k > i:
                            expr.addTerms(1, z[j, i, k])
                        elif j < k and k < i:
                            expr.addTerms(1, z[j, k, i])
                
                expr.addTerms(-1, phi[i])
                model.addLConstr(expr, GRB.EQUAL, 0, 'C15')
    else:  # proposed models
        if equality == True:  # model 3-1, ECC
            x_list = []
            z_list = []
            
            for i in range(N):
                x_list.append(i)
                for j in range(N):
                    for k in range(N):
                        if j > i and k > j:
                            z_list.append((i, j, k))
                        
            
            x = model.addVars(x_list, vtype=GRB.BINARY, name='x')
            z = model.addVars(z_list, lb=0.0, ub=1, vtype=GRB.CONTINUOUS, name='z')
            
            model.update()
                
            expr = gp.LinExpr()
            
            for i in range(N):
                expr.addTerms(r_i[i], x[i])
                
                for j in range(N):
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(1/(m-2)*(r_ij[i, j]+r_ij[i, k]+r_ij[j, k]), z[i, j, k])
                            expr.addTerms(r_ijk[i, j, k], z[i, j, k])
                            
            model.setObjective(expr, GRB.MAXIMIZE)
            
            # Constraints
            
            ## C8
            model.addLConstr(gp.quicksum(x[i] for i in range(N)) == m, 'C8')
            
                            
            ## C17
            for i in range(N):
                expr = gp.LinExpr()
                
                for j in range(N):
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(1, z[i, j, k])
                        elif i > j and k > i:
                            expr.addTerms(1, z[j, i, k])
                        elif k > j and i > k:
                            expr.addTerms(1, z[j, k, i])
                
                expr.addTerms(-(m-1)*(m-2)/2, x[i])
                model.addLConstr(expr, GRB.EQUAL, 0, f'C17_{i}')
                
            ## C20
            expr = gp.LinExpr()
            for i in range(N):
                expr.addTerms(d_i[i], x[i])
                
                for j in range(N): 
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(-1 / (m-2) * (d_ij[i, j]+d_ij[i, k]+d_ij[j, k]), z[i, j, k])
                            expr.addTerms(d_ijk[i, j, k], z[i, j, k])
            
            model.addLConstr(expr, GRB.LESS_EQUAL, b, 'C20')
        else:  # 3-2, IECC
            x_list = []
            z_list = []
            
            for i in range(N):
                x_list.append(i)
                for j in range(N):
                    for k in range(N):
                        if j > i and k > j:
                            z_list.append((i, j, k))
                        
            
            x = model.addVars(x_list, vtype=GRB.BINARY, name='x')
            z = model.addVars(z_list, lb=0.0, ub=1, vtype=GRB.CONTINUOUS, name='z')
            u = model.addVars(m-2 , vtype=GRB.BINARY, name='u')
            phi = model.addVars(N, lb=0.0, vtype=GRB.CONTINUOUS, name='phi')
            Q = model.addVars(N, lb=0.0, vtype=GRB.CONTINUOUS, name='Q')
            xi = model.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name='xi')
            zeta = model.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name='zeta')            
            bigM = 10 ** 9
            
            model.update()
                
            expr = gp.LinExpr()
            for i in range(N):
                expr.addTerms(r_i[i], x[i])
                
                for j in range(N):
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(r_ijk[i, j, k], z[i, j, k])
            
            expr.addTerms(1, xi)
            model.setObjective(expr, GRB.MAXIMIZE)
            
            # Constraints
            ## C21
            for t in range(m-2):
                expr = gp.LinExpr()
                expr.addTerms(bigM, u[t])
                expr.addConstant(-bigM)
                
                for i in range(N):
                    for j in range(N):                        
                        for k in range(N):
                            if j > i and k > j:
                                expr.addTerms(1/(t+1)*(r_ij[i, j] + r_ij[i, k] + r_ij[j, k]), z[i, j, k])
                expr.addTerms(-1, xi)
                model.addLConstr(expr, GRB.LESS_EQUAL, 0, f'C21_{t}')
                
            ## C22
            for t in range(m-2):
                expr = gp.LinExpr()
                expr.addConstant(bigM)
                expr.addTerms(-bigM, u[t])
                
                for i in range(N):
                    for j in range(N):                        
                        for k in range(N):
                            if j > i and k > j:
                                expr.addTerms(1/(t+1)*(r_ij[i, j] + r_ij[i, k] + r_ij[j, k]), z[i, j, k])
                expr.addTerms(-1, xi)
                model.addLConstr(expr, GRB.GREATER_EQUAL, 0, f'C22_{t}')
            
            ## C23
            for t in range(m-2):
                expr = gp.LinExpr()
                expr.addTerms(bigM, u[t])
                expr.addConstant(-bigM)
                
                for i in range(N):
                    for j in range(N):                        
                        for k in range(N):
                            if j > i and k > j:
                                expr.addTerms(1/(t+1)*(d_ij[i, j] + d_ij[i, k] + d_ij[j, k]), z[i, j, k])
                expr.addTerms(-1, zeta)
                model.addLConstr(expr, GRB.LESS_EQUAL, 0, f'C23_{t}')
                
            ## C24
            for t in range(m-2):
                expr = gp.LinExpr()
                expr.addConstant(bigM)
                expr.addTerms(-bigM, u[t])
                
                for i in range(N):
                    for j in range(N):                        
                        for k in range(N):
                            if j > i and k > j:
                                expr.addTerms(1/(t+1)*(d_ij[i, j] + d_ij[i, k] + d_ij[j, k]), z[i, j, k])
                expr.addTerms(-1, zeta)
                model.addLConstr(expr, GRB.GREATER_EQUAL, 0, f'C24_{t}')
            
            ## C26
            model.addLConstr(gp.quicksum(u[t] for t in range(m-2)) == 1, 'C26')
            
            ## C27
            model.addLConstr(gp.quicksum(x[i] for i in range(N)) == gp.quicksum((t+3) * u[t] for t in range(m-2)), 'C27')
            
            ## C28, C29, C30
            for i in range(N):
                model.addLConstr(phi[i] + Q[i] == (m-1) * (m-2) / 2 * (1-x[i]) + gp.quicksum((t+1) * (t+2) / 2 * u[t] for t in range(m-2)), f'C28_{i}')
                model.addLConstr(Q[i] <= (m-1) * (m-2) * (1-x[i]), f'C29_{i}')
                model.addLConstr(phi[i] <= (m-1) * (m-2) / 2 * x[i], f'C30_{i}')
            
            ## C32
            expr = gp.LinExpr()
            for i in range(N):
                expr.addTerms(d_i[i], x[i])
                for j in range(N):
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(d_ijk[i, j, k], z[i, j, k])
            
            expr.addTerms(-1, zeta)
            model.addLConstr(expr, GRB.LESS_EQUAL, b, f'C32')
                        
            
            ## C33
            for i in range(N):
                expr = gp.LinExpr()
                for j in range(N):                        
                    for k in range(N):
                        if j > i and k > j:
                            expr.addTerms(1, z[i, j, k])
                        elif j < i and k > i:
                            expr.addTerms(1, z[j, i, k])
                        elif j < k and k < i:
                            expr.addTerms(1, z[j, k, i])
                
                expr.addTerms(-1, phi[i])
                model.addLConstr(expr, GRB.EQUAL, 0, f'C33_{i}')
    
    model.optimize()
    
#     model.write('test.lp')

    if model.status == GRB.OPTIMAL:
        solution = model.getAttr('X', x)
#         print('optimal obj value:', model.objVal)
#         print('x:', solution)
    elif model.status == GRB.INF_OR_UNBD:
        print('Model is infeasible or unbounded')
    elif model.status == GRB.INFEASIBLE:
        print('Model is infeasible')
    elif model.status == GRB.UNBOUNDED:
        print('Model is unbounded')
    else:
        print('Optimization ended with status %d' % model.status)
    
    return model

In [3]:
def test_solver(N, m, r_i, r_ij, r_ijk, d_i, d_ij, d_ijk, b):
    model = gp.Model('PPSP')
    model.setParam('OutputFlag', False)  # mute
    # x_list = []
    # y_list = []
    z_list = []

    for i in range(N):
    # x_list.append(i)
        for j in range(N):
    #     if j > i:
    #         y_list.append((i, j))
            for k in range(N):
                if j > i and k > j:
                    z_list.append((i, j, k))


    # x = model.addVars(x_list, vtype=GRB.BINARY, name='x')
    # y = model.addVars(y_list, lb=0.0, ub=1, vtype=GRB.CONTINUOUS, name='y')
    z = model.addVars(z_list, vtype=GRB.BINARY, name='z')

    model.update()

    expr = gp.LinExpr()

    for i in range(N):
    # expr.addTerms(r_i[i], x[i])
        for j in range(N):
    #     if j > i:
    #         expr.addTerms(r_ij[i, j], y[i, j])
            for k in range(N):
                if j > i and k > j:
                    ## x
                    expr.addTerms(2/((m-1)*(m-2)) * (r_i[i]+r_i[j]+r_i[k]), z[i, j, k])

                    ## y
                    expr.addTerms(1/(m-2) * (r_ij[i, j]+r_ij[i, k]+r_ij[j, k]), z[i, j, k])

                    ## z
                    expr.addTerms(r_ijk[i, j, k], z[i, j, k])

    model.setObjective(expr, GRB.MAXIMIZE)

    # Constraints

    ## C7
    expr = gp.LinExpr()
    for i in range(N):
        for j in range(N):
            for k in range(N):
                if j > i and k > j:
                    ## x
                    expr.addTerms(2/((m-1)*(m-2)) * (d_i[i]+d_i[j]+d_i[k]), z[i, j, k])

                    ## y
                    expr.addTerms(-1/(m-2) * (d_ij[i, j]+d_ij[i, k]+d_ij[j, k]), z[i, j, k])

                    ## z
                    expr.addTerms(d_ijk[i, j, k], z[i, j, k])
    model.addLConstr(expr, GRB.LESS_EQUAL, b, 'resource')

    ## C8
    expr = gp.LinExpr()
    for i in range(N):
        for j in range(N):
            for k in range(N):
                if j > i and k > j:
                    ## z
                    expr.addTerms(1, z[i, j, k])
    model.addLConstr(expr, GRB.EQUAL, m*(m-1)*(m-2)/6, 'cardinatility')

    model.optimize()

    if model.status == GRB.OPTIMAL:
        solution = model.getAttr('X', z)
#         print('optimal obj value:', model.objVal)
#         print('z:', solution)
    elif model.status == GRB.INF_OR_UNBD:
        print('Model is infeasible or unbounded')
    elif model.status == GRB.INFEASIBLE:
        print('Model is infeasible')
    elif model.status == GRB.UNBOUNDED:
        print('Model is unbounded')
    else:
        print('Optimization ended with status %d' % model.status)
        
    return model

# Generate Instances
* $b \sim Uniform(0.05G, 0.1G),~ G=\sum_{i}d_i - \sum_{i}\sum_{j>i}d_{i, j} + \sum_{i}\sum_{j>i}\sum_{k>j}d_{i,j,k}$
* resource
    - $d_i \sim Uniform(1, 10)$
    - $d_{i, j} \sim Uniform(5, 20)$
    - $d_{i, j, k} \sim Uniform(10, 50)$
* benefit
    - $r_i \sim Uniform(10, 100)$
    - $r_{i, j} \sim Uniform(50, 200)$
    - $r_{i, j, k} \sim Uniform(100, 500)$

In [4]:
def instance_generator(N, m):
    instance = {'N': N, 'm': m}
    r_i = {}
    r_ij = {}
    r_ijk = {}

    d_i = {}
    d_ij = {}
    d_ijk = {}
    
    G = 0
    for i in range(N):
        r_i[i] = np.random.randint(10, 101)
        d_i[i] = np.random.randint(1, 11)
        G += d_i[i]
        for j in range(N):
            if j > i:
                r_ij[i, j] = np.random.randint(50, 201)
                d_ij[i, j] = np.random.randint(5, 21)
                G -= d_ij[i, j]
            for k in range(N):
                if j > i and k > j:
                    r_ijk[i, j, k] = np.random.randint(101, 501)
                    d_ijk[i, j, k] = np.random.randint(10, 51)
                    G += d_ijk[i, j, k]
                    
    b = np.random.randint(0.05*G, 0.1*G + 1)
    
    instance['r_i'] = r_i
    instance['r_ij'] = r_ij
    instance['r_ijk'] = r_ijk
    instance['d_i'] = d_i
    instance['d_ij'] = d_ij
    instance['d_ijk'] = d_ijk
    instance['b'] = b
    
    return instance

# Experiment

In [16]:
scale = ['small', 'medium', 'large']
instances = {}
instances['small'] = [instance_generator(16, 3) for _ in range(50)]
instances['medium'] = [instance_generator(32, 5) for _ in range(50)]
instances['large'] = [instance_generator(64, 7) for _ in range(50)]

In [17]:
result_3_1 = {}
result_test = {}

for s in scale:
    result_3_1[s] = []
    result_test[s] = []
    for instance in instances[s]:
        result_3_1[s].append(PPSP_solver(**instance, mode='proposed', equality=True))
        result_test[s].append(test_solver(**instance))

GurobiError: Out of memory

In [46]:
print('3-1' ,np.mean([item.Runtime for item in result_3_1['small']]))
print('test' ,np.mean([item.Runtime for item in result_test['small']]))

3-1 0.004275398254394531
test 0.0008003616333007813


In [47]:
print('3-1' ,np.mean([item.Runtime for item in result_3_1['medium']]))
print('test' ,np.mean([item.Runtime for item in result_test['medium']]))

3-1 0.02604167938232422
test 0.0031990432739257813


In [48]:
print('3-1' ,np.mean([item.ObjVal for item in result_3_1['large']]))
print('test' ,np.mean([item.ObjVal for item in result_test['large']]))

3-1 1302.54
test 1302.54


In [14]:
m = result_3_1['medium'][1]
for v in m.getVars():
    if v.x == 1:
        print('%s %g' % (v.varName, v.x))

print('Obj: %g' % m.objVal)

x[0] 1
x[4] 1
x[21] 1
x[25] 1
x[26] 1
z[0,4,21] 1
z[0,4,25] 1
z[0,4,26] 1
z[0,21,25] 1
z[0,21,26] 1
z[0,25,26] 1
z[4,21,25] 1
z[4,21,26] 1
z[4,25,26] 1
z[21,25,26] 1
Obj: 6207


In [15]:
m = result_test['medium'][2]
for v in m.getVars():
    if v.x == 1:
        print('%s %g' % (v.varName, v.x))

print('Obj: %g' % m.objVal)

z[0,6,28] 1
z[0,17,24] 1
z[2,8,17] 1
z[3,5,8] 1
z[3,28,31] 1
z[4,14,18] 1
z[4,21,23] 1
z[7,20,25] 1
z[7,24,26] 1
z[24,25,26] 1
Obj: 7024.33


# Test

In [9]:
N = 5
m = 4
r_i = {}
r_ij = {}
r_ijk = {}

d_i = {}
d_ij = {}
d_ijk = {}
b = 100

for i in range(N):
    r_i[i] = 0
    d_i[i] = 0
    for j in range(N):
        if j > i:
            r_ij[i, j] = 0
            d_ij[i, j] = 0
        for k in range(N):
            if j > i and k > j:
                r_ijk[i, j, k] = 0
                d_ijk[i, j, k] = 0
                
r_ij[0, 1] = 200
r_ij[2, 3] = 200
r_ijk[0, 2, 4] = 350

d_i[0] = 5000
d_ijk[0, 1, 2] = 5000
d_ijk[1, 2, 3] = 5000

In [10]:
PPSP_solver(N, m, r_i, r_ij, r_ijk, d_i, d_ij, d_ijk, b, mode='conventional', equality=True)

Model is infeasible


In [7]:
PPSP_solver(N, m, r_i, r_ij, r_ijk, d_i, d_ij, d_ijk, b, mode='proposed', equality=True)

Model is infeasible


In [8]:
PPSP_solver(N, m, r_i, r_ij, r_ijk, d_i, d_ij, d_ijk, b, mode='conventional', equality=False)

optimal obj value: 200.0
x: {0: 0.0, 1: 0.0, 2: 1.0, 3: 1.0, 4: 1.0}


In [12]:
PPSP_solver(N, m, r_i, r_ij, r_ijk, d_i, d_ij, d_ijk, b, mode='proposed', equality=False)

optimal obj value: 200.0
x: {0: 0.0, 1: -0.0, 2: 1.0, 3: 1.0, 4: 1.0}


# Higher Order

## From Prop 1, Prop 2
**<u>Corollary 1.</u>**
Let $m$ and $N$ be given positive integers such that $1 \leq m < N$ and assume that $x_i \in \{0, 1\}$ for $i = 1,
\ldots ,N$ satisfying $\sum\limits_{i=1}^N{x_{i}}=m$. For a set of non-negative variables $x^{[p]}_{i_1, i_2, \ldots i_p} \in [0, 1],~~ i_1, i_2, \ldots i_p \in \{1, \ldots ,N\}$ with all of the subscripts being distict, if  

$$\begin{align}
& \sum\limits_{i_1 < i_2}^N \sum\limits_{i_2<i_3}^N \cdots \sum\limits_{i_{p-1} ~ < i_p}^N {x^{[p]}_{i_1, i_2, \ldots i_p}} + \\
& \sum\limits_{i_2 < i_1}^N \sum\limits_{i_1<i_3}^N \cdots \sum\limits_{i_{p-1} ~ < i_p}^N {x^{[p]}_{i_2, i_1, \ldots i_p}} + \\
& \sum\limits_{i_2 < i_3}^N \sum\limits_{i_3<i_4}^N \cdots \sum\limits_{i_p < i_1}^N {x^{[p]}_{i_2, i_3, \ldots i_1}} = C^{m-1}_{p-1} x_{i_1},~ \text{for}~ i_1 = 1, \ldots ,N,
\end{align}$$

then $x_{i_1}x_{i_2} \ldots x_{i_p} = x^{[p]}_{i_1, i_2, \ldots i_p}$ with all of the subscripts of $x^{[p]}$ being distinct

## From Theorem 1

**<u>Corollary 2.</u>**
For a vector $x = (x_1, \ldots , x_N) \in \{0, 1\}^N$, one set of non-negative variables $x^{[p]}_{i_1, i_2, \ldots i_p} \in [0, 1],~~ i_1, i_2, \ldots i_p \in \{1, \ldots ,N\}$, and one set of non-negative variables $x^{[p+1]}_{i_1, i_2, \ldots i_{p+1}} \in [0, 1],~~ i_1, i_2, \ldots i_p, i_{p+1} \in \{1, \ldots ,N\}$ with all of the subscripts being distict and $p+1 \leq m < N$, where $m$ and $N$ are two integer variables. Let $r_{i_1, i_2, \ldots i_p}$ be the additional benefit from implementing project $i_1, i_2, \ldots ,i_p$ simultaneously. If the constraints from Corollary 1. with $p = p$ and $p = p +1$, and $\sum_{i=1}^N x_i = 1$ are satisfied, then  

$\sum\limits_{i_1 = 1}^N \sum\limits_{i_2 > i_1}^N \cdots \sum\limits_{i_{p} > ~ i_{p-1}}^N (r_{i_1, i_2, \ldots i_p}) {x^{[p]}_{i_1, i_2, \ldots i_p}} = \frac{1}{m-p}\sum\limits_{i_1 = 1}^N \sum\limits_{i_2 > i_1}^N \cdots \sum\limits_{i_p > ~ i_{p-1}}^N ~\sum\limits_{i_{p+1} ~ > i_p}^N (r_{i_1, i_2 \ldots i_p} + r_{i_1, i_3, \ldots i_{p+1}} + r_{i_1, i_4, \ldots i_{p+1}} + \cdots + r_{i_2, i_3, \ldots i_{p+1}})  x^{[p+1]}_{i_1, i_2, \ldots i_{p+1}}$

<!-- **Proof.** We examine the following cases:  
* Case 1: ($m = p+1$) In this case  -->

## Model Formulation (z only)

$\begin{align}
max ~ \hat{F}(z) = &\frac{1}{C^{m-1}_2}\sum\limits_{i = 1}^N \sum\limits_{j > i}^N \sum\limits_{k > j}^N (r_i+r_j+r_k)z_{ijk} + \\
& \frac{1}{C^{m-2}_1}\sum\limits_{i = 1}^N \sum\limits_{j > i}^N \sum\limits_{k > j}^N (r_{i,j}+r_{i,k}+r_{j, k})z_{ijk} + \\
& \sum\limits_{i = 1}^N \sum\limits_{j > i}^N \sum\limits_{k > j}^N r_{i,j,k}z_{ijk}
\end{align}$

$\begin{align}
s.t. ~~ & \sum\limits_{i = 1}^N \sum\limits_{j > i}^N \sum\limits_{k > j}^N z_{ijk} = C^m_3 \\
        & \frac{1}{C^{m-1}_2}\sum\limits_{i = 1}^N \sum\limits_{j > i}^N \sum\limits_{k > j}^N (d_i+d_j+d_k)z_{ijk} - 
          \frac{1}{C^{m-2}_1}\sum\limits_{i = 1}^N \sum\limits_{j > i}^N \sum\limits_{k > j}^N (d_{i,j}+d_{i,k}+d_{j, k})z_{ijk} + 
          \sum\limits_{i = 1}^N \sum\limits_{j > i}^N \sum\limits_{k > j}^N d_{i,j,k}z_{ijk} \leq b \\
        & z_{ijk} \in \{0, 1\}, ~ i,j,k \in 1, \ldots ,N
\end{align}$